In [45]:
import soundfile # to read audio file
import numpy as np
import pandas as pd
import librosa # to extract speech features
import glob
import os
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model, to_categorical

In [46]:
splits_length_in_seconds = 3 # how long is one video split (in seconds)
folder_containing_full_videos = "youtube-videos/Essence-of-Calculus" # all videos from this folder will be used to create dfs
data_output_folder_name = "prediction-data" # output folder for .csv files
model = keras.models.load_model("samuli_model") # model used for predictions

In [47]:
# dont change
emotions_dict ={
    "neutral": 0,
    "happy": 1,
    "angry": 2,
    "sad": 3,
    "fearful": 4
}

category_to_emotion = {}

for key in emotions_dict.keys():
    value = emotions_dict[key]
    category_to_emotion[value] = key

feature_used = {
    "mfcc": True,
    "chroma": True,
    "mel": True,
    "contrast": False,
    "tonnetz": False
}

# Create dfs and save them to csv

In [48]:
from pydub import AudioSegment
from pydub.utils import mediainfo
import math
import shutil
import matplotlib.pyplot as plt

In [49]:

def extract_feature(file_name):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
    """
    mfcc = feature_used["mfcc"]
    chroma = feature_used["chroma"]
    mel = feature_used["mel"]
    contrast = feature_used["contrast"]
    tonnetz = feature_used["tonnetz"]
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        if (len(X.shape) != 1):
            if (X.shape[1] == 2):
                X = X[:,0]
            else:
                return [None]
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result


## Create splits

In [50]:
def rm_folder(path):
    try:
        shutil.rmtree(path)
    except:
        None

In [51]:
splits_length_seconds = splits_length_in_seconds

for file in glob.glob(f"{folder_containing_full_videos}/*.mp3"):
        file_splits = []
        file_path = os.getcwd() + "\\" + file
        basename = os.path.basename(file)
        basename_without_ext = basename.split(".")[0]
        split_folder_name = f"{basename_without_ext}_split"
        sound = AudioSegment.from_file(file)
        
        parent_dir = os.getcwd() + f"\\{folder_containing_full_videos}"
        path_to_splits = os.path.join(parent_dir, split_folder_name)
        
        rm_folder(path_to_splits)
        os.mkdir(path_to_splits, 0o666)
                 
        i = 0
        split_length_ms = splits_length_seconds * 1000
        length = len(sound)
        splits_amount = math.ceil(length/split_length_ms)
        while (i < splits_amount):
            start_time = i * splits_length_seconds
            end_time = start_time + splits_length_seconds
            start_index = i * split_length_ms
            end_index = start_index + split_length_ms
            
            split = sound[start_index:end_index]
            split_video_name = f"{basename_without_ext}_split_{start_time}-{end_time}.wav"
            
            path = f"{folder_containing_full_videos}/{split_folder_name}/{split_video_name}"
            
            split.export(path, format='wav', bitrate="768000")
            i = i + 1

## Get dataframes


In [52]:
def get_dataframe_with_predictions(folder_containing_splits):
    
    folder_name = os.getcwd() + f"\\{folder_containing_full_videos}\\" + folder_containing_splits
    print("processing " + folder_name)
    df_emotions = pd.DataFrame(columns=['file_name', 'emotion', 'start_time', 'emotion_class', 'confidence'])

    for file in os.listdir(folder_name):
        file_name = os.fsdecode(file)
        path = folder_name + "\\" + file_name
        features = extract_feature(path)
        features = np.expand_dims(features, axis=1)
        features = np.expand_dims(features, axis=0)
        features = features.astype(np.float32)

        start_time = file_name.split('_')[len(file_name.split('_'))-1].split('-')[0]

        predic = model.predict(features)
        predic_confidence = float(np.amax(predic,1)[0])

        emotion = category_to_emotion[np.argmax(predic)]
        emotion_class = emotions_dict[emotion]


        new_row = {'file_name': file_name,
                   'emotion': emotion,
                  'start_time': int(start_time),
                  'emotion_class': emotion_class,
                  'confidence': predic_confidence}
        df_emotions = df_emotions.append(new_row, ignore_index=True)

    df_emotions.sort_values(by='start_time', inplace=True)
    return df_emotions

In [53]:
splits_dir = os.getcwd() + f"\\{folder_containing_full_videos}"

meta_data_dict = {}

for root, subdirectories, files in os.walk(splits_dir):
    for subdirectory in subdirectories:
        folder_name = splits_dir + f"\\{subdirectory}"
        df = get_dataframe_with_predictions(subdirectory)
        
        csv_name = '_'.join(subdirectory.split("_")[0:-1])
        df_path = data_output_folder_name + f"/{csv_name}.csv"
        print(df_path)
        df.to_csv(df_path, index=False)

processing D:\gitProjects\ids-project\data-analysis\youtube-videos/Essence-of-Calculus\Derivative formulas through geometry  Chapter 3 Essence of calculus_1949212_45000_413_split
prediction-data/Derivative formulas through geometry  Chapter 3 Essence of calculus_1949212_45000_413.csv
processing D:\gitProjects\ids-project\data-analysis\youtube-videos/Essence-of-Calculus\Higher order derivatives  Chapter 10 Essence of calculus_492881_10000_70_split
prediction-data/Higher order derivatives  Chapter 10 Essence of calculus_492881_10000_70.csv
processing D:\gitProjects\ids-project\data-analysis\youtube-videos/Essence-of-Calculus\Implicit differentiation whats going on here  Chapter 6 Essence of calculus_1249073_25000_309_split
prediction-data/Implicit differentiation whats going on here  Chapter 6 Essence of calculus_1249073_25000_309.csv
processing D:\gitProjects\ids-project\data-analysis\youtube-videos/Essence-of-Calculus\Integration and the fundamental theorem of calculus  Chapter 8 Essen